<a href="https://colab.research.google.com/github/kataria342/nlp_project2/blob/main/UpdatedCode4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from textblob import TextBlob
import nltk
#nltk.download('all')

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [5]:
symptomsDF1 = pd.read_csv('/workspaces/DiagnoSmart/Dataset/dataset.csv')
symptomsDF1 = symptomsDF1.replace(np.NaN, ' ')
cols = list(symptomsDF1.columns)
for i in cols[1:]:
  symptomsDF1[i] = symptomsDF1[i].str.split("_")

symptomsDF1['combined_symptoms'] = symptomsDF1['Symptom_1'] + symptomsDF1['Symptom_2'] + symptomsDF1['Symptom_3'] + symptomsDF1['Symptom_4'] + symptomsDF1['Symptom_5'] + symptomsDF1['Symptom_6'] + symptomsDF1['Symptom_7'] + symptomsDF1['Symptom_8'] + symptomsDF1['Symptom_9'] + symptomsDF1['Symptom_10'] + symptomsDF1['Symptom_11'] + symptomsDF1['Symptom_12'] + symptomsDF1['Symptom_13'] + symptomsDF1['Symptom_14'] + symptomsDF1['Symptom_15'] + symptomsDF1['Symptom_16'] + symptomsDF1['Symptom_17']
symptomsDF1.drop(columns=cols[1:],inplace=True)
symptomsDF1['combined_symptoms'] = symptomsDF1['combined_symptoms'].apply(lambda x: ' '.join(x))


symptomsDF1.head(2)

,Disease,combined_symptoms
0,Fungal infection,itching skin rash nodal skin eruptions dischromic patches
1,Fungal infection,skin rash nodal skin eruptions dischromic patches


In [6]:
sample_date = pd.read_csv('/workspaces/DiagnoSmart/Dataset/Medical Data.csv')

symptomsDF2 = sample_date[['Diesase_Name','symptoms_elements', 'symptoms_paragraph']]
symptomsDF2['combined_symptoms'] = symptomsDF2['symptoms_elements'] + symptomsDF2['symptoms_paragraph']
symptomsDF2.drop(columns=['symptoms_elements','symptoms_paragraph'], inplace=True)

symptomsDF2.dropna(inplace=True)
symptomsDF2.columns = ['Disease','combined_symptoms']


data = [symptomsDF2, symptomsDF1]
symptomsDF = pd.concat(data, ignore_index=True)
print(symptomsDF.size)
symptomsDF.drop_duplicates(inplace=True)
print(symptomsDF.size)
symptomsDF.head(2)
symptomsDF = symptomsDF.groupby('Disease')['combined_symptoms'].agg(' '.join).reset_index()
symptomsDF[symptomsDF['Disease'] == 'Fungal infection']


12734
2102


,Disease,combined_symptoms
296,Fungal infection,itching skin rash nodal skin eruptions dischromic patches skin rash nodal skin eruptions dischromic patches itching nodal skin eruptions dischromic patches itching skin rash dischromic patches itching skin rash nodal skin eruptions


In [7]:
def clean_duplicate(text):
  words = text.split()
  unique_words = set()
  result = []
  for word in words:
      if word not in unique_words:
          unique_words.add(word)
          result.append(word)

  return ' '.join(result)

symptomsDF['combined_symptoms'] = symptomsDF['combined_symptoms'].apply(clean_duplicate)


symptomsDF[symptomsDF['Disease'] == 'Fungal infection']

,Disease,combined_symptoms
296,Fungal infection,itching skin rash nodal eruptions dischromic patches


In [8]:
symptomsDF[(symptomsDF['Disease'] == 'Cholestasis of pregnancy') | (symptomsDF['Disease'] == 'Chronic cholestasis') | (symptomsDF['Disease'] == 'Cholangiocarcinoma (bile duct cancer)')]

,Disease,combined_symptoms
156,Cholangiocarcinoma (bile duct cancer),"Yellowing of your skin and the whites eyes (jaundice)Intensely itchy skinWhite-colored stoolsFatigueAbdominal pain on right side, just below ribsLosing weight without tryingFeverNight sweatsDark urineSigns symptoms cholangiocarcinoma include:"
158,Cholestasis of pregnancy,"Yellowing of the skin and whites eyes, called jaundiceNauseaLoss appetiteOily, foul-smelling stoolsIntense itching is main symptom cholestasis pregnancy. But there no rash. Typically, you feel itchy on palms your hands or soles feet, but may everywhere. The often worse at night bother so much that can't sleep."
159,Chronic cholestasis,itching vomiting yellowish skin nausea loss of appetite abdominal pain yellowing eyes


In [9]:
def cleandata1(text):
    tag_mapping = {
    'NN': 'n',
    'NNS': 'n',
    'NNP': 'n',
    'NNPS': 'n',
    'VB': 'v',
    'VBD': 'v',
    'VBG': 'v',
    'VBN': 'v',
    'VBP': 'v',
    'VBZ': 'v',
    'JJ': 'a',
    'JJR': 'a',
    'JJS': 'a',
    'RB': 'r',
    'RBR': 'r',
    'RBS': 'r',
    'WRB':'r'
    }
    text = "".join([word.lower() for word in re.sub(r"[^a-zA-Z]", " ", text)])
    tokens = word_tokenize(text)
    text = [word for word in tokens if len(word)>2 or word not in stop_words]
    text = " ".join([lemmatizer.lemmatize(word, pos='n' if tag_mapping.get(TextBlob(word).tags[0][1]) == None else tag_mapping.get(TextBlob(word).tags[0][1])) for word in text])
    return text
symptomsDF['combined_symptoms'] = symptomsDF['combined_symptoms'].apply(lambda x: cleandata1(x))

symptomsDF.head(2)

,Disease,combined_symptoms
0,(vertigo) Paroymsal Positional Vertigo,vomit headache nausea spin movement loss balance unsteadiness
1,ACL injury,loud pop pop sensation the kneesevere pain and inability continue activityrapid swellingloss range motiona feel instability give way with weight bearingsigns symptom acl injury usually include


In [10]:
diseaseDescriptionDF1 = pd.read_csv('/workspaces/DiagnoSmart/Dataset/symptom_Description.csv')
diseaseDescriptionDF2 = sample_date[['Diesase_Name','overview']]
diseaseDescriptionDF2.columns = ['Disease','Description']
data = [diseaseDescriptionDF2, diseaseDescriptionDF1]
diseaseDescriptionDF = pd.concat(data, ignore_index=True)
diseaseDescriptionDF.drop_duplicates(inplace=True)
diseaseDescriptionDF.set_index('Disease', inplace = True)
diseaseDescriptionDF.head(2)

,Description
Disease,
Atrial fibrillation,"In a typical heart, a tiny group of cells at the sinus node sends out an electrical signal. The signal travels through the heart's upper chambers to the atrioventricular (AV) node. The signal then passes into the lower heart chambers, causing them to squeeze and pump out blood. In atrial fibrillation, electrical signals fire from many locations in the upper chambers, causing them to beat chaotically. Since the AV node doesn't prevent all of these chaotic signals from entering the lower chambers, the heart beats faster and irregularly."
Abdominal aortic aneurysm,"An abdominal aortic aneurysm occurs when a lower part of the body's main artery, called the aorta, becomes weakened and bulges."


In [149]:
# def CorrectSpellings(text):
#     tokens = [str(TextBlob(word).correct()) for word in word_tokenize(text)]
#     text = " ".join(tokens)
#     return text

# diseaseDescriptionDF['Disease'] = diseaseDescriptionDF['Disease'].apply(lambda X: CorrectSpellings(X))
# symptomsDF['Disease'] = symptomsDF['Disease'].apply(lambda X: CorrectSpellings(X))

In [11]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
embed = model(symptomsDF['combined_symptoms'])

2024-01-29 10:01:20.453241: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 10:01:22.854304: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 10:01:22.854404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 10:01:23.230505: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 10:01:24.025420: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 10:01:24.027577: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [12]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=1)

neigh.fit(embed)

NearestNeighbors(n_neighbors=1)

In [13]:
# import joblib
# joblib.dump(neigh, 'knn_model.joblib')

['knn_model.joblib']

In [15]:
symptomsDF.to_csv('CleanedSymptoms.csv')
diseaseDescriptionDF.to_csv('CleanedDescripition.csv')


In [13]:
symptoms = input('enter your symptoms: \n')
print('Symptoms Entered: ', symptoms)
symptoms = cleandata1(symptoms)

emd = model([symptoms])
nei = neigh.kneighbors(emd,return_distance=False)[0][0]

disease_name = symptomsDF['Disease'].iloc[nei]
description = diseaseDescriptionDF['Description'].loc[disease_name]

print('**********************************')
print('Predicted Disease : ',disease_name.title())
print('Disease Definition:',description)
print('**********************************')

Symptoms Entered:  I have skin rashes since morning
have skin rash since morning
**********************************
Predicted Disease :  Fungal Infection
Disease Definition: In humans, fungal infections occur when an invading fungus takes over an area of the body and is too much for the immune system to handle. Fungi can live in the air, soil, water, and plants. There are also some fungi that live naturally in the human body. Like many microbes, there are helpful fungi and harmful fungi.
**********************************


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_matrix = cosine_similarity(embed, emd)

In [15]:
max = cosine_similarity_matrix[0]
index = 0
for i in range(1,len(cosine_similarity_matrix)):
    if cosine_similarity_matrix[i] > max:
        max = cosine_similarity_matrix[i]
        index = i

disease_name = symptomsDF['Disease'].iloc[index]
description = diseaseDescriptionDF['Description'].loc[disease_name]

print('**********************************')
print('Predicted Disease : ',disease_name.title())
print('Disease Definition:',description)
print('**********************************')

**********************************
Predicted Disease :  Fungal Infection
Disease Definition: In humans, fungal infections occur when an invading fungus takes over an area of the body and is too much for the immune system to handle. Fungi can live in the air, soil, water, and plants. There are also some fungi that live naturally in the human body. Like many microbes, there are helpful fungi and harmful fungi.
**********************************
